<a href="https://colab.research.google.com/github/azhar2205/paddle-ball-using-dqlearn/blob/master/Monthly_Retracement_from_Options_Breakeven_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README

Download spot data for current year from https://www.niftyindices.com/reports/historical-data and download at /content/drive/MyDrive/institution_activity_analysis/data/raw/NIFTY_50

Download expiry dates from https://www.nseindia.com/api/historical/fo/derivatives/meta?&from=01-01-2021&to=31-12-2021&instrumentType=OPTIDX&symbol=NIFTY

# INPUT

In [ ]:
# README - Paste token from Kite "Authorization" header
KITE_TOKEN = "enctoken 05BgZL45XBg+vcg+V4JRhsogqPd6pzhIBV8Ql229XUvYQt94dejV3HkCC1g4wOaNDJ6EYbAP9seQCe12/0ehMg1NRLDig/9xI3eJhIGal6CZUHx/h2oJnQ=="
###

FROM_DATE = '2021-12-31' # Day 1 of expiry for start_date
TO_DATE = '2024-12-31' # Expiry last day for end_date

# Imports

In [ ]:
import datetime
from IPython.display import display
import itertools
import json
import math
from more_itertools import windowed
import numpy as np
import os
import pandas as pd
import pathlib
import requests
import seaborn as sns
import shutil
from tqdm.notebook import tqdm
import zipfile

# Constants

In [ ]:
SCRIPT_PATH = "/content/drive/MyDrive/institution_activity_analysis"
DATA_PATH = os.path.join(SCRIPT_PATH, "data")
RAW_DATA_PATH = os.path.join(DATA_PATH, "raw")
FO_CSV_PATH = "/content"
FO_ZIP_PATH = os.path.join(RAW_DATA_PATH, "FO", "ZIP")
SPOT_CSV_PATH = os.path.join(RAW_DATA_PATH, "NIFTY_50")
TODAY = datetime.date.today()
YDAY = TODAY - datetime.timedelta(1)

In [ ]:
KITE_HEADERS = {
    "authority": "kite.zerodha.com",
    "accept": "*/*",
    "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,mr;q=0.7,hi;q=0.6",
    "authorization": f"{KITE_TOKEN}",
    "referer": "https://kite.zerodha.com/static/build/chart-beta.html?v=3.3.2",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# Variables

In [ ]:
HOLIDAY_LIST = []
EXPIRY_DF = None
SPOT_DF = None

# Functions

In [ ]:
def unzip_file(zip_filepath, target_folderpath):
    with zipfile.ZipFile(zip_filepath, "r") as zip_ref:
        test_res = zip_ref.testzip()
        if test_res:
            raise RuntimeError(f"Error: Archive {zip_filepath} is bad. Error: {test_res}")
        else:
            zip_ref.extractall(target_folderpath)
#

def get_filename_for_date(n_date):
    yyyy = n_date.strftime("%Y")
    mm = n_date.strftime("%m")
    dd = n_date.strftime("%d")
    mon = n_date.strftime("%b").upper()
    dt_2024_07_08 = datetime.datetime.strptime("2024-07-08", "%Y-%m-%d").date()

    # Before 2024-07-08: fo19AUG2022bhav.csv.zip
    # After 2024-07-08: BhavCopy_NSE_FO_0_0_0_20240708_F_0000.csv.zip
    fo_filename = f"BhavCopy_NSE_FO_0_0_0_{yyyy}{mm}{dd}_F_0000.csv.zip"
    if n_date < dt_2024_07_08:
        fo_filename = f"fo{dd}{mon}{yyyy}bhav.csv.zip"
    return { "fo_filename": fo_filename }
#

In [ ]:
def load_holidays():
    return pd.read_csv(os.path.join(DATA_PATH, "holidays.txt"), header=None)[0].tolist()
#

def is_holiday(n_date):
    date_str = n_date.strftime("%d") + "-" + n_date.strftime("%b") + "-" + n_date.strftime("%Y")
    # weekday() -> (5=Sat, 6=Sun)
    return n_date.weekday() >= 5 or date_str in HOLIDAY_LIST
#

def load_expiries():
    df = pd.read_csv(os.path.join(DATA_PATH, "expiries.csv"))
    df["expiry"] = df["expiry"].apply(pd.to_datetime)
    df["alt_expiry"] = df["alt_expiry"].apply(pd.to_datetime)
    df = df.sort_values("expiry")
    return df
#

def get_expiries_df(mm, yyyy):
    return EXPIRY_DF[(EXPIRY_DF["expiry"].dt.month == mm) & (EXPIRY_DF["expiry"].dt.year == yyyy)]
#

def get_expiries(mm, yyyy):
    return get_expiries_df(mm, yyyy).sort_values("expiry", ascending=True)
#

def get_previous_month(mm, yyyy):
    if mm > 1:
        return mm-1, yyyy
    else:
        return 12, yyyy-1
#

def get_next_month(mm, yyyy):
    if mm < 12:
        return mm+1, yyyy
    else:
        return 1, yyyy+1
#

def get_monthly_expiry(mm, yyyy):
    return get_expiries(mm, yyyy)["expiry"].tolist()[-1]
#

def get_monthly_alt_expiry(mm, yyyy):
    return get_expiries(mm, yyyy)["alt_expiry"].tolist()[-1]
#

def get_previous_monthly_expiry(mm, yyyy):
    prev_mm, prev_yyyy = get_previous_month(mm, yyyy)
    return get_monthly_expiry(prev_mm, prev_yyyy)
#

def read_spot_data():
    dfs = list()
    files = pathlib.Path(SPOT_CSV_PATH).glob("*.csv")
    for f in files:
        data = pd.read_csv(f)
        dfs.append(data)
    #
    df = pd.concat(dfs, ignore_index=True)
    df["Date"] = df["Date"].apply(pd.to_datetime)
    df = df.set_index(["Date"])
    return df
#


In [ ]:
def call_kite_api(url, retry=False):
    try:
        response = requests.get(url, headers=KITE_HEADERS, timeout=10)
        if not response.ok:
            print(f"ERROR: {url} failed with status code {response.status_code} and body {response.text}")
            response.raise_for_status()
        response_json = json.loads(response.text)
        if response_json["status"] != "success":
            raise ValueError(f"{url} failed")
        return response_json["data"]["candles"]
    except (requests.ReadTimeout, json.JSONDecodeError) as e:
        if retry:
            print("Retrying after 5 sec...")
            time.sleep(5)
            return call_kite_api(url, retry=False)
        else:
            raise e
#

def get_instruments_historical_url(instrument_id, from_date, to_date):
    return f"https://kite.zerodha.com/oms/instruments/historical/{instrument_id}/day?user_id=JV5904&oi=0&from={from_date}&to={to_date}"
#

def build_spot_df_from_kite_data(kite_data, from_date, to_date):
    df = pd.DataFrame(kite_data, columns=["Date", "Open", "High", "Low", "Close", "Volume"])
    df = df.drop(columns="Volume")
    df["Date"] = df["Date"].apply(pd.to_datetime)
    df = df[(df["Date"] >= from_date) & (df["Date"] <= to_date)]
    df = df.drop_duplicates(subset=["Date"])
    df["Date"] = df["Date"].dt.date
    df["Date"] = df["Date"].apply(pd.to_datetime)
    df = df.set_index(["Date"])
    return df
#

def download_nifty_data_from_kite(from_date, to_date):
    nifty_instrument_id = "256265"
    kite_url = get_instruments_historical_url(nifty_instrument_id, from_date, to_date)
    nifty_data = call_kite_api(kite_url, retry=True)
    return build_spot_df_from_kite_data(nifty_data, from_date, to_date)
#

In [ ]:
def read_fo_df(filepath, n_date):
    fo_df = pd.read_csv(filepath, header=0)
    dt_2024_07_08 = datetime.datetime.strptime("2024-07-08", "%Y-%m-%d").date()
    if n_date >= dt_2024_07_08:
        fo_df = fo_df[["TckrSymb", "XpryDt", "StrkPric", "OptnTp", "OpnPric", "ClsPric"]]
        fo_df = fo_df.rename(columns={
            "TckrSymb": "symbol",
            "XpryDt": "expiry",
            "StrkPric": "strike",
            "OptnTp": "option_type",
            "OpnPric": "open",
            "ClsPric": "close",
        })
        fo_df = fo_df[fo_df["symbol"] == "NIFTY"].copy()
        fo_df["option_type"] = fo_df["option_type"].fillna("FUT")
        fo_df["strike"] = fo_df["strike"].fillna(0)
    else:
        fo_df = fo_df[["SYMBOL", "EXPIRY_DT", "STRIKE_PR", "OPTION_TYP", "OPEN", "CLOSE"]]
        fo_df = fo_df.rename(columns={
            "SYMBOL": "symbol",
            "EXPIRY_DT": "expiry",
            "STRIKE_PR": "strike",
            "OPTION_TYP": "option_type",
            "OPEN": "open",
            "CLOSE": "close",
        })
        fo_df = fo_df[fo_df["symbol"] == "NIFTY"].copy()
        fo_df["expiry"] = fo_df["expiry"].apply(lambda x: datetime.datetime.strptime(x, "%d-%b-%Y").date().strftime("%Y-%m-%d"))
        fo_df["option_type"] = fo_df["option_type"].apply(lambda x: "FUT" if x == "XX" else x)
    # end if
    fo_df["strike"] = fo_df["strike"].astype(int)
    return fo_df
#

def get_atm_strike_for_spot(spot):
    if spot % 50 > 25:
        return int(spot - (spot % 50) + 50)
    else:
        return int(spot - (spot % 50))
#

def get_ce_hedge_strike(ce_strike_price, ce_strike):
    ce_breakeven = ce_strike + ce_strike_price
    if ce_breakeven % 50 > 25:
        return int(ce_breakeven - (ce_breakeven % 50) + 50)
    else:
        return int(ce_breakeven - (ce_breakeven % 50))
#

def get_pe_hedge_strike(pe_strike_price, pe_strike):
    pe_breakeven = pe_strike - pe_strike_price
    if pe_breakeven % 50 < 25:
        return int(pe_breakeven - (pe_breakeven % 50))
    else:
        return int(pe_breakeven - (pe_breakeven % 50) + 50)
#

def get_strike_price(fno_df, expiry, alt_expiry, atm_strike, option_type, open_close="close"):
    result = fno_df[
        (fno_df["expiry"] == expiry)
        & (fno_df["strike"] == atm_strike)
        & (fno_df["option_type"] == option_type)
    ][open_close]
    if not len(result):
        result = fno_df[
            (fno_df["expiry"] == alt_expiry)
            & (fno_df["strike"] == atm_strike)
            & (fno_df["option_type"] == option_type)
        ][open_close]
    return result.values[0]
#


In [ ]:
def read_data_for_month(mm, yyyy):
  is_first_day = True
  is_trade_started = be_at_low = be_at_high = False
  exp_open_atm_ce_price = exp_open_atm_pe_price = None
  exp_open_atm_ce_strike = exp_open_atm_pe_strike = None
  break_even = None
  trade_open_atm_ce_strike = trade_open_atm_pe_strike = None
  trade_open_atm_ce_price = trade_open_atm_pe_price = None
  trade_open_hedge_ce_price = trade_open_hedge_pe_price = None
  mtm_df = pd.DataFrame(data=[], columns=["CE_day1", "PE_day1", "stdl_day1", "stdl_day1_tgt", "stdl_day1_sl", "CE_be", "PE_be", "stdl_be", "stdl_be_tgt", "stdl_be_sl", "dir_be", "dir_be_tgt", "dir_be_sl", "stdl_day1_stdl_be", "stdl_day1_stdl_be_tgt", "stdl_day1_stdl_be_sl", "stdl_day1_dir_be", "stdl_day1_dir_be_tgt", "stdl_day1_dir_be_sl"])
  last_expiry = get_previous_monthly_expiry(mm, yyyy)
  curr_expiry = get_monthly_expiry(mm, yyyy)
  alt_curr_expiry = get_monthly_alt_expiry(mm, yyyy)
  curr_expiry_str = curr_expiry.date().strftime("%Y-%m-%d")
  alt_curr_expiry_str = None
  if not pd.isna(alt_curr_expiry):
    alt_curr_expiry_str = alt_curr_expiry.date().strftime("%Y-%m-%d")
  start_date = last_expiry.date() + datetime.timedelta(days=1)
  end_date = min(curr_expiry.date(), datetime.date.today() - datetime.timedelta(days=1))
  n_days = (end_date - start_date).days
  # print(f"Reading data for past {n_days} days for {mm:>02}/{yyyy} expiry...")
  for n_date in (start_date + datetime.timedelta(n) for n in range(0, n_days + 1)):
    if is_holiday(n_date):
      continue
    n_date_str = n_date.strftime("%Y-%m-%d")
    mtm_df.loc[n_date_str] = [0.0, 0.0, 0.0, '', '', 0.0, 0.0, 0.0, '', '', 0.0, '', '', 0.0, '', '', 0.0, '', '']
    filenames = get_filename_for_date(n_date)
    fo_csvfilepath = os.path.join(FO_CSV_PATH, filenames["fo_filename"][:-4])
    if not os.path.isfile(fo_csvfilepath):
      fo_zipfilepath = os.path.join(FO_ZIP_PATH, filenames["fo_filename"])
      if not os.path.isfile(fo_zipfilepath):
        raise RuntimeError(f'Error: FO ZIP file not found for date: {n_date_str}')
      unzip_file(fo_zipfilepath, FO_CSV_PATH)
      if not os.path.isfile(fo_csvfilepath):
        raise RuntimeError(f'Error: FO CSV file not found for date: {n_date_str}')
    fo_df = read_fo_df(fo_csvfilepath, n_date)
    # Note first day ATM SPOT, CE and PE strikes
    if is_first_day:
      spot = round(SPOT_DF.at[n_date_str, "Open"])
      atm_strike = get_atm_strike_for_spot(spot)
      exp_open_atm_ce_strike = atm_strike
      exp_open_atm_pe_strike = atm_strike
    exp_open_atm_ce_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, exp_open_atm_ce_strike, "CE", open_close="open")
    exp_open_atm_pe_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, exp_open_atm_pe_strike, "PE", open_close="open")
    mtm_df.at[n_date_str, "CE_day1"] = exp_open_atm_ce_price_curr
    mtm_df.at[n_date_str, "PE_day1"] = exp_open_atm_pe_price_curr
    # Note first day ATM CE and PE prices
    if is_first_day:
      # break_even = 80% of ATM CE + PE straddle premium
      # break_even = (exp_open_atm_ce_price_curr + exp_open_atm_pe_price_curr) * .8
      break_even = (exp_open_atm_ce_price_curr + exp_open_atm_pe_price_curr)
      exp_open_atm_ce_price = exp_open_atm_ce_price_curr
      exp_open_atm_pe_price = exp_open_atm_pe_price_curr
    stdl_on_day_1_mtm = exp_open_atm_ce_price - exp_open_atm_ce_price_curr + exp_open_atm_pe_price - exp_open_atm_pe_price_curr
    mtm_df.at[n_date_str, "stdl_day1"] = stdl_on_day_1_mtm
    mtm_df.at[n_date_str, "stdl_day1_stdl_be"] = stdl_on_day_1_mtm
    mtm_df.at[n_date_str, "stdl_day1_dir_be"] = stdl_on_day_1_mtm
    if stdl_on_day_1_mtm >= (exp_open_atm_ce_price + exp_open_atm_pe_price) / 3:
      mtm_df.at[n_date_str, "stdl_day1_tgt"] = 'T'
      if not is_trade_started:
        mtm_df.at[n_date_str, "stdl_day1_stdl_be_tgt"] = 'T'
        mtm_df.at[n_date_str, "stdl_day1_dir_be_tgt"] = 'T'
    elif stdl_on_day_1_mtm <= -1 * (exp_open_atm_ce_price + exp_open_atm_pe_price) / 3:
      mtm_df.at[n_date_str, "stdl_day1_sl"] = 'S'
      if not is_trade_started:
        mtm_df.at[n_date_str, "stdl_day1_stdl_be_sl"] = 'S'
        mtm_df.at[n_date_str, "stdl_day1_dir_be_sl"] = 'S'
    if not is_trade_started \
    and (
        exp_open_atm_ce_price_curr >= break_even
        or exp_open_atm_pe_price_curr >= break_even
    ):
      spot = round(SPOT_DF.at[n_date_str, "Close"])
      trade_open_atm_ce_strike = get_atm_strike_for_spot(spot)
      trade_open_atm_pe_strike = get_atm_strike_for_spot(spot)
      trade_open_atm_ce_price = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_ce_strike, "CE")
      trade_open_atm_pe_price = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_pe_strike, "PE")
      trade_open_hedge_ce_price = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_ce_strike + 200, "CE")
      trade_open_hedge_pe_price = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_pe_strike - 200, "PE")
      # Market is going up
      if exp_open_atm_ce_price_curr >= break_even:
        be_at_high = True
      # Market is going down
      elif exp_open_atm_pe_price_curr >= break_even:
        be_at_low = True
      is_trade_started = True
    # Compute MTM
    if is_trade_started:
      trade_open_atm_ce_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_ce_strike, "CE")
      trade_open_atm_pe_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_pe_strike, "PE")
      trade_open_hedge_ce_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_ce_strike + 200, "CE")
      trade_open_hedge_pe_price_curr = get_strike_price(fo_df, curr_expiry_str, alt_curr_expiry_str, trade_open_atm_pe_strike - 200, "PE")
      mtm_df.at[n_date_str, "CE_be"] = trade_open_atm_ce_price_curr
      mtm_df.at[n_date_str, "PE_be"] = trade_open_atm_pe_price_curr
      if be_at_high:
        # dir_at_be_mtm = trade_open_atm_ce_price - trade_open_atm_ce_price_curr - trade_open_hedge_ce_price + trade_open_hedge_ce_price_curr
        dir_at_be_mtm = trade_open_atm_ce_price - trade_open_atm_ce_price_curr
        mtm_df.at[n_date_str, "dir_be"] = dir_at_be_mtm
        stdl_day1_dir_be_mtm = stdl_on_day_1_mtm + dir_at_be_mtm
        mtm_df.at[n_date_str, "stdl_day1_dir_be"] = stdl_day1_dir_be_mtm
        if dir_at_be_mtm >= trade_open_atm_ce_price / 3:
          mtm_df.at[n_date_str, "dir_be_tgt"] = 'T'
        elif dir_at_be_mtm <= -1 * trade_open_atm_ce_price / 3:
          mtm_df.at[n_date_str, "dir_be_sl"] = 'S'
        if stdl_day1_dir_be_mtm >= (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_ce_price) / 3:
          mtm_df.at[n_date_str, "stdl_day1_dir_be_tgt"] = 'T'
        elif stdl_day1_dir_be_mtm <= -1 * (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_ce_price) / 3:
          mtm_df.at[n_date_str, "stdl_day1_dir_be_sl"] = 'S'
      elif be_at_low:
        # dir_at_be_mtm = trade_open_atm_pe_price - trade_open_atm_pe_price_curr - trade_open_hedge_pe_price + trade_open_hedge_pe_price_curr
        dir_at_be_mtm = trade_open_atm_pe_price - trade_open_atm_pe_price_curr
        mtm_df.at[n_date_str, "dir_be"] = dir_at_be_mtm
        stdl_day1_dir_be_mtm = stdl_on_day_1_mtm + dir_at_be_mtm
        mtm_df.at[n_date_str, "stdl_day1_dir_be"] = stdl_day1_dir_be_mtm
        if dir_at_be_mtm >= trade_open_atm_pe_price / 3:
          mtm_df.at[n_date_str, "dir_be_tgt"] = 'T'
        elif dir_at_be_mtm <= -1 * trade_open_atm_pe_price / 3:
          mtm_df.at[n_date_str, "dir_be_sl"] = 'S'
        if stdl_day1_dir_be_mtm >= (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_pe_price) / 3:
          mtm_df.at[n_date_str, "stdl_day1_dir_be_tgt"] = 'T'
        elif stdl_day1_dir_be_mtm <= -1 * (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_pe_price) / 3:
          mtm_df.at[n_date_str, "stdl_day1_dir_be_sl"] = 'S'
      #
      stdl_at_be_mtm = trade_open_atm_ce_price - trade_open_atm_ce_price_curr + trade_open_atm_pe_price - trade_open_atm_pe_price_curr
      mtm_df.at[n_date_str, "stdl_be"] = stdl_at_be_mtm
      stdl_day1_stdl_be_mtm = stdl_on_day_1_mtm + stdl_at_be_mtm
      mtm_df.at[n_date_str, "stdl_day1_stdl_be"] = stdl_day1_stdl_be_mtm
      if stdl_at_be_mtm >= (trade_open_atm_ce_price + trade_open_atm_pe_price) / 3:
        mtm_df.at[n_date_str, "stdl_be_tgt"] = 'T'
      elif stdl_at_be_mtm <= -1 * (trade_open_atm_ce_price + trade_open_atm_pe_price) / 3:
        mtm_df.at[n_date_str, "stdl_be_sl"] = 'S'
      if stdl_day1_stdl_be_mtm >= (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_ce_price + trade_open_atm_pe_price) / 3:
        mtm_df.at[n_date_str, "stdl_day1_stdl_be_tgt"] = 'T'
      elif stdl_day1_stdl_be_mtm <= -1 * (exp_open_atm_ce_price + exp_open_atm_pe_price + trade_open_atm_ce_price + trade_open_atm_pe_price) / 3:
        mtm_df.at[n_date_str, "stdl_day1_stdl_be_sl"] = 'S'
      #
    # Reset first day flag
    if is_first_day:
      is_first_day = False
  # end for
  display(mtm_df)
  # mtm_df = mtm_df.reset_index()
  # return mtm_df

  # stdl_on_day_1_T_S = get_tgt_or_sl(mtm_df, "stdl_day1")
  # stdl_on_day_1_T = get_tgt(mtm_df, "stdl_day1")
  # stdl_on_day_1_S = get_sl(mtm_df, "stdl_day1")
  # stdl_at_be_T_S = get_tgt_or_sl(mtm_df, "stdl_be")
  # stdl_at_be_T = get_tgt(mtm_df, "stdl_be")
  # stdl_at_be_S = get_sl(mtm_df, "stdl_be")
  # dir_at_be_T_S = get_tgt_or_sl(mtm_df, "dir_be")
  # dir_at_be_T = get_tgt(mtm_df, "dir_be")
  # dir_at_be_S = get_sl(mtm_df, "dir_be")
  # return pd.Series([stdl_on_day_1_T_S, stdl_on_day_1_T, stdl_on_day_1_S, stdl_at_be_T_S, stdl_at_be_T, stdl_at_be_S, dir_at_be_T_S, dir_at_be_T, dir_at_be_S], name=f'{yyyy}/{mm:>02}')

  # stdl_day1_dir_be_T_S = get_tgt_or_sl(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_dir_be_T = get_tgt(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_dir_be_S = get_sl(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_stdl_be_T_S = get_tgt_or_sl(mtm_df, "stdl_day1_stdl_be")
  # stdl_day1_stdl_be_T = get_tgt(mtm_df, "stdl_day1_stdl_be")
  # stdl_day1_stdl_be_S = get_sl(mtm_df, "stdl_day1_stdl_be")
  # return pd.Series([stdl_day1_dir_be_T_S, stdl_day1_dir_be_T, stdl_day1_dir_be_S, stdl_day1_stdl_be_T_S, stdl_day1_stdl_be_T, stdl_day1_stdl_be_S], name=f'{yyyy}/{mm:>02}')

  # stdl_on_day_1_last_day = get_last_day_pnl(mtm_df, "stdl_day1")
  # stdl_on_day_1_max = get_max_pnl(mtm_df, "stdl_day1")
  # stdl_on_day_1_min = get_min_pnl(mtm_df, "stdl_day1")
  # stdl_day1_dir_be_last_day = get_last_day_pnl(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_dir_be_max = get_max_pnl(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_dir_be_min = get_min_pnl(mtm_df, "stdl_day1_dir_be")
  # stdl_day1_stdl_be_last_day = get_last_day_pnl(mtm_df, "stdl_day1_stdl_be")
  # stdl_day1_stdl_be_max = get_max_pnl(mtm_df, "stdl_day1_stdl_be")
  # stdl_day1_stdl_be_min = get_min_pnl(mtm_df, "stdl_day1_stdl_be")
  # return pd.Series([stdl_on_day_1_last_day, stdl_on_day_1_max, stdl_on_day_1_min, stdl_day1_dir_be_last_day, stdl_day1_dir_be_max, stdl_day1_dir_be_min, stdl_day1_stdl_be_last_day, stdl_day1_stdl_be_max, stdl_day1_stdl_be_min], name=f'{yyyy}/{mm:>02}')
#

def get_tgt_or_sl(mtm_df, prefix):
  is_tgt_na = pd.isna(mtm_df.index[mtm_df[f'{prefix}_tgt'] == 'T'].min())
  is_sl_na = pd.isna(mtm_df.index[mtm_df[f'{prefix}_sl'] == 'S'].min())
  if is_tgt_na and is_sl_na:
    last_day_pnl = mtm_df[prefix].tolist()[-1]
    if last_day_pnl >= 0:
      return 'T'
    else:
      return 'S'
  elif is_sl_na:
    return 'T'
  elif is_tgt_na:
    return 'S'
  elif mtm_df.index[mtm_df[f'{prefix}_tgt'] == 'T'].min() < mtm_df.index[mtm_df[f'{prefix}_sl'] == 'S'].min():
    return 'T'
  else:
    return 'S'

def get_tgt(mtm_df, prefix):
  return not mtm_df[mtm_df[f'{prefix}_tgt'] == 'T'].empty

def get_sl(mtm_df, prefix):
  return not mtm_df.index[mtm_df[f'{prefix}_sl'] == 'S'].empty

def get_last_day_pnl(mtm_df, prefix):
  return mtm_df[prefix].tolist()[-1]

def get_max_pnl(mtm_df, prefix):
  return max(mtm_df[prefix].tolist())

def get_min_pnl(mtm_df, prefix):
  return min(mtm_df[prefix].tolist())

read_data_for_month(9, 2023)

,CE_day1,PE_day1,stdl_day1,stdl_day1_tgt,stdl_day1_sl,CE_be,PE_be,stdl_be,stdl_be_tgt,stdl_be_sl,dir_be,dir_be_tgt,dir_be_sl,stdl_day1_stdl_be,stdl_day1_stdl_be_tgt,stdl_day1_stdl_be_sl,stdl_day1_dir_be,stdl_day1_dir_be_tgt,stdl_day1_dir_be_sl
2023-09-01,320.00,157.95,0.00,,,0.00,0.00,0.00,,,0.00,,,0.00,,,0.00,,
2023-09-04,383.00,95.80,-0.85,,,0.00,0.00,0.00,,,0.00,,,-0.85,,,-0.85,,
2023-09-05,429.85,72.45,-24.35,,,0.00,0.00,0.00,,,0.00,,,-24.35,,,-24.35,,
2023-09-06,455.55,55.10,-32.70,,,0.00,0.00,0.00,,,0.00,,,-32.70,,,-32.70,,
2023-09-07,421.00,58.35,-1.40,,,0.00,0.00,0.00,,,0.00,,,-1.40,,,-1.40,,
2023-09-08,595.00,39.95,-157.00,,,213.05,138.75,0.00,,,0.00,,,-157.00,,,-157.00,,
2023-09-11,663.05,26.00,-211.10,,S,319.55,83.15,-50.90,,,-106.50,,S,-262.00,,,-317.60,,S
2023-09-12,925.70,20.70,-468.45,,S,331.65,83.25,-63.10,,,-118.60,,S,-531.55,,S,-587.05,,S
2023-09-13,803.05,28.00,-353.10,,S,391.75,66.90,-106.85,,,-178.70,,S,-459.95,,S,-531.80,,S
2023-09-14,966.35,20.70,-509.10,,S,417.40,45.05,-110.65,,,-204.35,,S,-619.75,,S,-713.45,,S


# Main

In [ ]:
HOLIDAY_LIST = load_holidays()
EXPIRY_DF = load_expiries()
SPOT_DF = download_nifty_data_from_kite(FROM_DATE, TO_DATE)

In [ ]:
def run_for_all():
  sr_arr = []
  for yyyy in range(2022, 2025):
    for mm in range(1, 13):
      sr = read_data_for_month(yyyy=yyyy, mm=mm)
      sr_arr.append(sr)
  df = pd.concat(sr_arr, axis=1).T
  # df.columns = ['stdl_day1_T_S', 'stdl_day1_T', 'stdl_day1_S', 'stdl_be_T_S', 'stdl_be_T', 'stdl_be_S', 'dir_be_T_S', 'dir_be_T', 'dir_be_S']
  # df.columns = ['stdl_day1_stdl_be_T_S', 'stdl_day1_stdl_be_T', 'stdl_day1_stdl_be_S', 'stdl_day1_dir_be_T_S', 'stdl_day1_dir_be_T', 'stdl_day1_dir_be_S']
  df.columns = ['stdl_day1_last_day', 'stdl_day1_max', 'stdl_day1_min', 'stdl_day1_stdl_be_last_day', 'stdl_day1_stdl_be_max', 'stdl_day1_stdl_be_min', 'stdl_day1_dir_last_day', 'stdl_day1_dir_be_max', 'stdl_day1_dir_be_min']
  display(df)
#

run_for_all()

,stdl_on_day_1_last_day,stdl_on_day_1_max,stdl_on_day_1_min,stdl_day1_stdl_be_last_day,stdl_day1_stdl_be_max,stdl_day1_stdl_be_min,stdl_day1_dir_last_day,stdl_day1_dir_be_max,stdl_day1_dir_be_min
2022/01,339.10,339.10,-455.00,640.45,640.45,-625.10,66.90,313.65,-449.50
2022/02,394.90,566.50,-9.50,394.90,566.50,-9.50,394.90,566.50,-9.50
2022/03,75.30,368.05,-85.50,75.30,368.05,-85.50,75.30,368.05,-85.50
2022/04,398.95,398.95,-119.80,742.25,742.25,-119.80,316.65,445.85,-147.45
2022/05,-579.50,1.90,-844.05,-514.70,1.90,-1143.55,-189.85,1.90,-886.10
2022/06,271.80,380.55,-189.10,527.95,624.15,-189.10,319.70,355.20,-189.10
2022/07,-317.95,252.40,-317.95,-590.00,252.40,-590.00,-451.55,252.40,-451.55
2022/08,20.95,264.65,-288.55,246.70,433.65,-378.50,329.20,512.60,-219.45
2022/09,105.80,422.50,0.00,105.80,422.50,0.00,105.80,422.50,0.00
2022/10,-238.90,186.75,-238.90,-321.75,311.15,-321.75,-12.45,368.00,-125.45


In [ ]:
def run_for_all():
  sr_arr = []
  for yyyy in range(2022, 2025):
    for mm in range(1, 13):
      sr = read_data_for_month(yyyy=yyyy, mm=mm)
      sr_arr.append(sr)
  df = pd.concat(sr_arr, axis=1).T
  df.columns = ['stdl_day1_T_S', 'stdl_day1_T', 'stdl_day1_S', 'stdl_be_T_S', 'stdl_be_T', 'stdl_be_S', 'dir_be_T_S', 'dir_be_T', 'dir_be_S']
  # df.columns = ['stdl_day1_stdl_be_T_S', 'stdl_day1_stdl_be_T', 'stdl_day1_stdl_be_S', 'stdl_day1_dir_be_T_S', 'stdl_day1_dir_be_T', 'stdl_day1_dir_be_S']
  # df.columns = ['stdl_day1_last_day', 'stdl_day1_max', 'stdl_day1_min', 'stdl_day1_stdl_be_last_day', 'stdl_day1_stdl_be_max', 'stdl_day1_stdl_be_min', 'stdl_day1_dir_last_day', 'stdl_day1_dir_be_max', 'stdl_day1_dir_be_min']
  display(df)
#

run_for_all()

,stdl_day1_T_S,stdl_day1_T,stdl_day1_S,stdl_be_T_S,stdl_be_T,stdl_be_S,dir_be_T_S,dir_be_T,dir_be_S
2022/01,S,True,True,T,True,True,S,True,True
2022/02,T,True,False,T,False,False,T,False,False
2022/03,T,True,False,T,False,False,T,False,False
2022/04,T,True,False,S,False,True,T,True,False
2022/05,S,False,True,T,True,False,S,False,True
2022/06,T,True,False,T,False,False,T,True,False
2022/07,S,False,True,T,True,True,S,True,True
2022/08,S,True,True,T,True,False,S,True,True
2022/09,T,True,False,T,False,False,T,False,False
2022/10,S,False,False,T,True,False,T,True,False


In [ ]:
def run_for_all():
  sr_arr = []
  for yyyy in range(2022, 2025):
    for mm in range(1, 13):
      sr = read_data_for_month(yyyy=yyyy, mm=mm)
      sr_arr.append(sr)
  df = pd.concat(sr_arr, axis=1).T
  # df.columns = ['stdl_day1_T_S', 'stdl_day1_T', 'stdl_day1_S', 'stdl_be_T_S', 'stdl_be_T', 'stdl_be_S', 'dir_be_T_S', 'dir_be_T', 'dir_be_S']
  df.columns = ['stdl_day1_stdl_be_T_S', 'stdl_day1_stdl_be_T', 'stdl_day1_stdl_be_S', 'stdl_day1_dir_be_T_S', 'stdl_day1_dir_be_T', 'stdl_day1_dir_be_S']
  # df.columns = ['stdl_day1_last_day', 'stdl_day1_max', 'stdl_day1_min', 'stdl_day1_stdl_be_last_day', 'stdl_day1_stdl_be_max', 'stdl_day1_stdl_be_min', 'stdl_day1_dir_last_day', 'stdl_day1_dir_be_max', 'stdl_day1_dir_be_min']
  display(df)
#

run_for_all()

,stdl_day1_stdl_be_T_S,stdl_day1_stdl_be_T,stdl_day1_stdl_be_S,stdl_day1_dir_be_T_S,stdl_day1_dir_be_T,stdl_day1_dir_be_S
2022/01,S,True,True,S,False,True
2022/02,T,True,False,T,True,False
2022/03,T,True,False,T,True,False
2022/04,T,True,False,T,False,False
2022/05,S,False,True,S,False,True
2022/06,T,True,False,T,False,False
2022/07,S,False,True,S,False,True
2022/08,S,True,True,T,True,False
2022/09,T,True,False,T,True,False
2022/10,S,False,False,S,False,False


## 2022

### Jan 2022

In [ ]:
read_data_for_month(1, 2022)

### Feb 2022

In [ ]:
read_data_for_month(2, 2022)

### Mar 2022

In [ ]:
read_data_for_month(3, 2022)

### Apr 2022

In [ ]:
read_data_for_month(4, 2022)

### May 2022

In [ ]:
read_data_for_month(5, 2022)

### Jun 2022

In [ ]:
read_data_for_month(6, 2022)

### Jul 2022

In [ ]:
read_data_for_month(7, 2022)

### Aug 2022

In [ ]:
read_data_for_month(8, 2022)

### Sep 2022

In [ ]:
read_data_for_month(9, 2022)

### Oct 2022

In [ ]:
read_data_for_month(10, 2022)

### Nov 2022

In [ ]:
read_data_for_month(11, 2022)

### Dec 2022

In [ ]:
read_data_for_month(12, 2022)

## 2023

### Jan 2023

In [ ]:
read_data_for_month(1, 2023)

### Feb 2023

In [ ]:
read_data_for_month(2, 2023)

### Mar 2023

In [ ]:
read_data_for_month(3, 2023)

### Apr 2023

In [ ]:
read_data_for_month(4, 2023)

### May 2023

In [ ]:
read_data_for_month(5, 2023)

### Jun 2023

In [ ]:
read_data_for_month(6, 2023)

### Jul 2023

In [ ]:
read_data_for_month(7, 2023)

### Aug 2023

In [ ]:
read_data_for_month(8, 2023)

### Sep 2023

In [ ]:
read_data_for_month(9, 2023)

### Oct 2023

In [ ]:
read_data_for_month(10, 2023)

### Nov 2023

In [ ]:
read_data_for_month(11, 2023)

### Dec 2023

In [ ]:
read_data_for_month(12, 2023)

## 2024

### Jan 2024

In [ ]:
read_data_for_month(1, 2024)

### Feb 2024

In [ ]:
read_data_for_month(2, 2024)

### Mar 2024

In [ ]:
read_data_for_month(3, 2024)

### Apr 2024

In [ ]:
read_data_for_month(4, 2024)

### May 2024

In [ ]:
read_data_for_month(5, 2024)

### Jun 2024

In [ ]:
read_data_for_month(6, 2024)

### Jul 2024

In [ ]:
read_data_for_month(7, 2024)

### Aug 2024

In [ ]:
read_data_for_month(8, 2024)

### Sep 2024

In [ ]:
read_data_for_month(9, 2024)

### Oct 2024

In [ ]:
read_data_for_month(10, 2024)